In [1]:
"""
Adapted from keras example cifar10_cnn.py
Train ResNet-18 on the CIFAR10 small images dataset.

GPU run command with Theano backend (with TensorFlow, the GPU is automatically used):
    THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python cifar10.py
"""
from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
import numpy as np
import pandas
import resnet
from PIL import Image
from skimage import io
import os
import sys

Using TensorFlow backend.


# load data

In [4]:
# data dir
directory = "./lfw_only_face_resize/"
train_x = []
train_y = []
test_x = []
test_y = []
threhold = 2
count = 0
for _dir in os.listdir(directory):
#     只訓練>5張的dataset
    people_image_count = len(os.listdir(os.path.join(directory, _dir)))
    if(people_image_count >= threhold):
        count += 1
        for index, filename in enumerate(os.listdir(os.path.join(directory, _dir))):
            if filename.endswith(".jpg"):
                img_path = os.path.join(directory, _dir, filename)
                image = Image.open(img_path)
#                 每個人取1張當作testing
                if(index == 0):
                    test_x.append(np.array(image))
                    test_y.append(_dir)
                else:
                    train_x.append(np.array(image))
                    train_y.append(_dir)

In [5]:
train_x, train_y = np.array(train_x), np.array(train_y)
test_x, test_y = np.array(test_x), np.array(test_y)
train_x, test_x = train_x.astype('float32'), test_x.astype('float32')
train_y, test_y = pandas.get_dummies(train_y).values, pandas.get_dummies(test_y).values

In [6]:
train_x /= 255
test_x /= 255

# model setting

In [7]:
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.001, patience=10)
csv_logger = CSVLogger('resnet64_faceValidation.csv')
# classes = os.listdir(directory)

In [8]:
batch_size = 32
nb_classes = count
nb_epoch = 100000
data_augmentation = False


In [9]:
# input image dimensions
img_rows, img_cols = 104, 96
# The CIFAR10 images are RGB.
img_channels = 3

In [10]:
model = resnet.ResnetBuilder.build_resnet_64((img_channels, img_rows, img_cols), nb_classes)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [11]:
train_x.shape

(7484, 104, 96, 3)

In [ ]:
train_y.shape

(7484, 1680)

In [ ]:
model.fit(train_x, train_y,
          batch_size=batch_size,
          epochs=nb_epoch,
          validation_data=(test_x, test_y),
          shuffle=True,
          verbose=1,
          callbacks=[lr_reducer, early_stopper, csv_logger])
# verbose=1,


Train on 7484 samples, validate on 1680 samples
Epoch 1/100000
7484/7484 [==============================] - 143s 19ms/step - loss: 12.4892 - acc: 0.0655 - val_loss: 18.5901 - val_acc: 0.0000e+00
Epoch 2/100000
7484/7484 [==============================] - 51s 7ms/step - loss: 10.2683 - acc: 0.0677 - val_loss: 10.7239 - val_acc: 5.9524e-04
Epoch 3/100000
7484/7484 [==============================] - 51s 7ms/step - loss: 9.5096 - acc: 0.0699 - val_loss: 10.4921 - val_acc: 0.0012
Epoch 4/100000
7484/7484 [==============================] - 51s 7ms/step - loss: 8.9729 - acc: 0.0712 - val_loss: 11.8103 - val_acc: 5.9524e-04
Epoch 5/100000
7484/7484 [==============================] - 51s 7ms/step - loss: 8.0366 - acc: 0.0742 - val_loss: 9.7056 - val_acc: 5.9524e-04
Epoch 6/100000
7484/7484 [==============================] - 54s 7ms/step - loss: 7.3576 - acc: 0.0776 - val_loss: 9.6588 - val_acc: 0.0024
Epoch 7/100000
7484/7484 [==============================] - 53s 7ms/step - loss: 6.8365 - acc:

In [ ]:
# save
model.save('my_model.h5')   # HDF5 file, you have to pip3 install h5py if don't have it

In [ ]:
# print('test before save: ', model.predict(X_test[0:2]))


In [ ]:
score = model.evaluate(x_test, y_test, batch_size=128)